In [1]:
import findspark
findspark.init()
findspark.find()

from pyspark.sql import SparkSession
spark = SparkSession.builder\
        .master("local")\
        .appName("Colab")\
        .config('spark.ui.port', '4050')\
        .getOrCreate()

In [2]:
from pyspark.ml.feature import OneHotEncoder, StringIndexer, VectorIndexer, VectorAssembler
from pyspark.ml.classification import DecisionTreeClassifier, RandomForestClassifier
from pyspark.ml.evaluation import MulticlassClassificationEvaluator
from pyspark.ml import Pipeline

from sklearn.metrics import confusion_matrix, precision_score, recall_score

In [10]:
# Đọc dữ liệu từ csv
data = spark.read.load('Data/mushrooms.csv', format="csv", header=True, delimiter=",", inferSchema=True)
data.show(5)

+-----+---------+-----------+---------+-------+----+---------------+------------+---------+----------+-----------+----------+------------------------+------------------------+----------------------+----------------------+---------+----------+-----------+---------+-----------------+----------+-------+
|class|cap-shape|cap-surface|cap-color|bruises|odor|gill-attachment|gill-spacing|gill-size|gill-color|stalk-shape|stalk-root|stalk-surface-above-ring|stalk-surface-below-ring|stalk-color-above-ring|stalk-color-below-ring|veil-type|veil-color|ring-number|ring-type|spore-print-color|population|habitat|
+-----+---------+-----------+---------+-------+----+---------------+------------+---------+----------+-----------+----------+------------------------+------------------------+----------------------+----------------------+---------+----------+-----------+---------+-----------------+----------+-------+
|    p|        x|          s|        n|      t|   p|              f|           c|        n|   

In [11]:
# Chia tập train, test
(train, test) = data.randomSplit([0.8, 0.2])

## Decision Tree

In [16]:
# Các cột dùng làm features
# columns = ['cap-shape', 'cap-surface', 'cap-color', 'bruises',\
#                'odor', 'gill-attachment', 'gill-spacing', 'gill-size',\
#                'gill-color', 'stalk-shape', 'stalk-root',\
#                'stalk-surface-above-ring', 'stalk-surface-below-ring',\
#                'stalk-color-above-ring', 'stalk-color-below-ring',\
#                'veil-type', 'veil-color', 'ring-number', 'ring-type',\
#                'spore-print-color', 'population', 'habitat']
columns = data.columns
columns.remove('class')

# Tên các cột dùng làm features sau khi được indexed
feature_columns = [col + '_indexed' for col in columns]

# Tạo các StringIndexer cho các cột categorical
indexers = [StringIndexer(inputCol=column, outputCol=column+"_indexed") for column in columns]

# Tạo assembler gom các giá trị ở các cột dùng làm features thành các vector ở cột "features"
assembler = VectorAssembler(inputCols=feature_columns, outputCol='features')

# Tạo indexer để tạo index cho các vector ở cột "features"
featureIndexer = VectorIndexer(inputCol="features", outputCol="indexedFeatures", maxCategories=50)

# Tạo indexer để map các giá trị cột "Label" sang chỉ số nhãn (label indices)
labelIndexer = StringIndexer(inputCol='class', outputCol='indexedLabel')

# Tạo một bộ phân loại decision tree
dt = DecisionTreeClassifier(featuresCol="indexedFeatures", labelCol="indexedLabel")

# Xây dựng pipeline là chuỗi các stringIndexers assembler, indexer và classifier
pipeline = Pipeline(stages=[*indexers, assembler, featureIndexer, labelIndexer, dt])   # *indexers: decompose indexers list

# Training với pipeline đã tạo
model = pipeline.fit(train)

In [17]:
# Sử dụng model đã train để dự đoán trên tập test
predictions = model.transform(test)
# Xem kết quả dự đoán gồm cột features, giá trị dự đoán và nhãn
predictions.select("features", "prediction", "indexedLabel").show(5, truncate=False)

+--------------------------------------------------------------------------------------+----------+------------+
|features                                                                              |prediction|indexedLabel|
+--------------------------------------------------------------------------------------+----------+------------+
|(22,[0,1,2,6,8,9,10,12,17,20,21],[3.0,2.0,1.0,1.0,4.0,1.0,1.0,1.0,1.0,3.0,1.0])       |0.0       |0.0         |
|(22,[0,1,2,6,8,9,10,12,17,20,21],[3.0,2.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,3.0,1.0])       |0.0       |0.0         |
|(22,[0,1,2,6,8,9,10,17,20,21],[3.0,2.0,1.0,1.0,2.0,1.0,1.0,1.0,2.0,1.0])              |0.0       |0.0         |
|(22,[0,1,2,6,8,9,10,11,12,17,20,21],[3.0,2.0,4.0,1.0,4.0,1.0,1.0,1.0,1.0,1.0,2.0,1.0])|0.0       |0.0         |
|(22,[0,1,2,6,8,9,10,17,20,21],[3.0,2.0,4.0,1.0,4.0,1.0,1.0,1.0,3.0,1.0])              |0.0       |0.0         |
+--------------------------------------------------------------------------------------+--------

In [18]:
# Tạo evaluator và tính độ chính xác dự đoán của model
evaluator = MulticlassClassificationEvaluator(
    labelCol="indexedLabel", predictionCol="prediction", metricName="accuracy")
accuracy = evaluator.evaluate(predictions)

# In thông tin summary của model
treeModel = model.stages[-1]
print(treeModel)

# In các kết quả đánh giá model
print("Decision Tree - Test Accuracy = %g" % (accuracy))
print("Decision Tree - Test Error = %g" % (1.0 - accuracy))

DecisionTreeClassificationModel: uid=DecisionTreeClassifier_56dfefe119cc, depth=4, numNodes=9, numClasses=2, numFeatures=22
Decision Tree - Test Accuracy = 0.997487
Decision Tree - Test Error = 0.00251256


In [19]:
# Tạo y_true, y_pred kiểu list để đánh giá model sử dụng các hàm của sklearn
y_true = predictions.select("indexedLabel").rdd.flatMap(lambda x: x).collect()
y_pred = predictions.select("prediction").rdd.flatMap(lambda x: x).collect()

# Tính toán confusion matrix, precision và recall của model
confusionmatrix = confusion_matrix(y_true, y_pred)
precision = precision_score(y_true, y_pred, average='micro')
recall = recall_score(y_true, y_pred, average='micro')

print("The Confusion Matrix for Decision Tree Model is :\n" + str(confusionmatrix))
print("The precision score for Decision Tree Model is: " + str(precision))
print("The recall score for Decision Tree Model is: " + str(recall))

The Confusion Matrix for Decision Tree Model is :
[[821   0]
 [  4 767]]
The precision score for Decision Tree Model is: 0.9974874371859297
The recall score for Decision Tree Model is: 0.9974874371859297


## Random Forest

In [20]:
# Tạo các StringIndexer cho các cột categorical
indexers = [StringIndexer(inputCol=column, outputCol=column+"_indexed") for column in columns]

# Tạo assembler gom các giá trị ở các cột dùng làm features thành các vector ở cột "features"
assembler = VectorAssembler(inputCols=feature_columns, outputCol="features")

# StringIndexer để mã hóa cột label thành một cột chỉ số nhãn.
# Theo mặc định, các nhãn được gán theo tần số. Vì vậy, 
# label xuất hiện thường xuyên nhất nhận được chỉ số 0.
labelIndexer = StringIndexer(inputCol="class", outputCol="indexedLabel")

# Tự động phân loại các features thành maxCategories nhóm.
featureIndexer = VectorIndexer(inputCol="features", outputCol="indexedFeatures", maxCategories=4)

# Tạo một bộ phân loại random forest
rf = RandomForestClassifier(labelCol="indexedLabel", featuresCol="indexedFeatures", numTrees=10)

# Xây dựng pipeline
pipeline = Pipeline(stages=[*indexers, assembler, labelIndexer, featureIndexer, rf])

# Train model.
model = pipeline.fit(train)

In [21]:
# Sử dụng model đã train để dự đoán trên tập test
predictions = model.transform(test)
# Chọn một vài cột để thấy được kết quả
predictions.select("features", "prediction", "indexedLabel").show(5, truncate=False)

+--------------------------------------------------------------------------------------+----------+------------+
|features                                                                              |prediction|indexedLabel|
+--------------------------------------------------------------------------------------+----------+------------+
|(22,[0,1,2,6,8,9,10,12,17,20,21],[3.0,2.0,1.0,1.0,4.0,1.0,1.0,1.0,1.0,3.0,1.0])       |0.0       |0.0         |
|(22,[0,1,2,6,8,9,10,12,17,20,21],[3.0,2.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,3.0,1.0])       |0.0       |0.0         |
|(22,[0,1,2,6,8,9,10,17,20,21],[3.0,2.0,1.0,1.0,2.0,1.0,1.0,1.0,2.0,1.0])              |0.0       |0.0         |
|(22,[0,1,2,6,8,9,10,11,12,17,20,21],[3.0,2.0,4.0,1.0,4.0,1.0,1.0,1.0,1.0,1.0,2.0,1.0])|0.0       |0.0         |
|(22,[0,1,2,6,8,9,10,17,20,21],[3.0,2.0,4.0,1.0,4.0,1.0,1.0,1.0,3.0,1.0])              |0.0       |0.0         |
+--------------------------------------------------------------------------------------+--------

In [22]:
# Đánh giá thuật toán 
evaluator = MulticlassClassificationEvaluator(
    labelCol="indexedLabel", predictionCol="prediction", metricName="accuracy")

accuracy = evaluator.evaluate(predictions)
print("Random Forest - Test Accuracy = %g" % (accuracy))
print("Random Forest - Test Error = %g" % (1.0 - accuracy))

Random Forest - Test Accuracy = 0.998744
Random Forest - Test Error = 0.00125628


In [23]:
# Tạo y_true, y_pred kiểu list để đánh giá model sử dụng các hàm của sklearn
y_true = predictions.select("indexedLabel").rdd.flatMap(lambda x: x).collect()
y_pred = predictions.select("prediction").rdd.flatMap(lambda x: x).collect()

# Tính toán confusion matrix, precision và recall của model
confusionmatrix = confusion_matrix(y_true, y_pred)
precision = precision_score(y_true, y_pred, average='micro')
recall = recall_score(y_true, y_pred, average='micro')

print("The Confusion Matrix for Random Forest Model is :\n" + str(confusionmatrix))
print("The precision score for Random Forest Model is: " + str(precision))
print("The recall score for Random Forest Model is: " + str(recall))

The Confusion Matrix for Random Forest Model is :
[[821   0]
 [  2 769]]
The precision score for Random Forest Model is: 0.9987437185929648
The recall score for Random Forest Model is: 0.9987437185929648
